# Checkpoint 1

A partir do dataset enviado, responda:

## Importação e Análise Inicial

1. Importe o arquivo CSV.(0,33)
2. Quantas linhas tem o dataset?(0,33)
3. Quantas colunas tem o dataset?(0,33)
4. Quais colunas têm o tipo `object`?(0,34)
5. Quais colunas têm o tipo `int64`?(0,34)
6. Existe alguma coluna de outro tipo?(0,33)

## Análise Estatística

7. Qual foi o total, o máximo, o mínimo do objetivo financeiro de todos os projetos (goal)?(0,66)
8. Qual o total, o máximo, o mínimo e o valor que representa 25% do dinheiro levantado (pledged)?(0,67)
9. Qual o total, o máximo, o mínimo e o valor que representa 75% dos apoiadores dos projetos(backers)?(0,67)

## Análise de Categorias

10. Qual a `category` mais presente? E quantos valores únicos ela possui?(0,5)
11. Qual a `main_category` mais presente? E quantos valores únicos ela possui?(0,5)
12. Qual a `currency` mais presente? E quantos valores únicos ela possui?(0,5)
13. Execute o processo de dummy na coluna `country`.(0,5)

## Análise de Valores Nulos

14. Qual a quantidade de valores nulos em cada feature?(0,66)
15. Qual a proporção de valores nulos por valores não nulos em cada feature?(0,66)
16. Faça o tratamento desses nulos e explique por que escolheu cada método.(0,68)

## Visualização Gráfica

17. Visualização Gráfica da feature `state`.(2)



----------------------------------------
🧩 Desafio: ETL de Dados Públicos

🎯 Objetivo
Construir um pipeline ETL no Apache Airflow que:

Extrai dados de algum dataset público

Transforma os dados limpando, filtrando e unificando informações.

Carrega o resultado em um banco de dados PostgreSQL (ou salva como Parquet).

Agenda a execução diária do pipeline (simulando ingestão incremental).

# Checkpoint 1

A partir do dataset enviado, responda:

## Importação e Análise Inicial

1. Importe o arquivo CSV.(0,33)
2. Quantas linhas tem o dataset?(0,33)
3. Quantas colunas tem o dataset?(0,33)
4. Quais colunas têm o tipo `object`?(0,34)
5. Quais colunas têm o tipo `int64`?(0,34)
6. Existe alguma coluna de outro tipo?(0,33)

In [37]:
import pandas as pd
import numpy as np
import ast

# Gerenciamento de experimentos com MLflow
import mlflow

# Download do dataset de filmes do Kaggle
#!kaggle datasets download -d rounakbanik/the-movies-dataset -p ./movies_dataset

# Descompactar o arquivo baixado
#!unzip -o ./movies_dataset/the-movies-dataset.zip -d ./movies_dataset


# Carregar o dataset de filmes usando o motor 'python' para evitar o ParserError
try:
    df = pd.read_csv("./movies_dataset/movies_metadata.csv", low_memory=False)
    print("O dataset foi importado com sucesso!")

    # Obter o número de linhas e colunas do DataFrame
    numero_linhas, numero_colunas = df.shape
    print(f"O dataset tem {numero_linhas} linhas")
    print(f"O dataset tem {numero_colunas} colunas.")

    colunas_object= df.select_dtypes(include=['object'])
    colunas_int64 = df.select_dtypes(include=['int64'])
    colunas_outrotipo = df.select_dtypes(exclude=['object', 'int64'])
    print(f"As colunas do tipo 'object' são: {colunas_object.columns.tolist()}")
    print(f"As colunas do tipo 'int64' são: {colunas_int64.columns.tolist()}")
    print(f"As colunas de outro tipo são: {colunas_outrotipo.columns.tolist()}")
    
except Exception as e:
    print(f"Ocorreu um erro ao carregar o CSV: {e}")
    print("Verifique se o arquivo 'mymoviedb.csv' existe no diretório './movies_dataset' e não está corrompido.")


O dataset foi importado com sucesso!
O dataset tem 45466 linhas
O dataset tem 24 colunas.
As colunas do tipo 'object' são: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'spoken_languages', 'status', 'tagline', 'title', 'video']
As colunas do tipo 'int64' são: []
As colunas de outro tipo são: ['revenue', 'runtime', 'vote_average', 'vote_count']


## Análise Estatística

7. Qual foi o total, o máximo, o mínimo do objetivo financeiro de todos os projetos (goal)?(0,66)
8. Qual o total, o máximo, o mínimo e o valor que representa 25% do dinheiro levantado (pledged)?(0,67)
9. Qual o total, o máximo, o mínimo e o valor que representa 75% dos apoiadores dos projetos(backers)?(0,67)

In [34]:
try:

    # --- Limpeza e Conversão da Coluna 'budget' ---
    # Converte a coluna 'budget' para um tipo numérico.
    # Erros de conversão (valores não numéricos) serão transformados em NaN (Not a Number).
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com orçamento maior que zero.
    filmes_com_budget = df[df['budget'] > 0].copy()

    # --- Análise Estatística ---
    total_budget = filmes_com_budget['budget'].sum()
    max_budget = filmes_com_budget['budget'].max()
    min_budget = filmes_com_budget['budget'].min()

    print(f"Total do orçamento de todos os filmes: ${total_budget:,.2f}")
    print(f"Maior orçamento entre os filmes: ${max_budget:,.2f}")
    print(f"Menor orçamento entre os filmes: ${min_budget:,.2f}")


    # --- Limpeza e Conversão da Coluna 'revenue' ---
    # A coluna 'revenue' também vem como texto e precisa ser convertida para número.
    # Usamos 'errors='coerce'' para transformar valores inválidos em NaN (Not a Number).
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com receita maior que zero.
    filmes_com_revenue = df[df['revenue'] > 0].copy()

    # --- Análise Estatística ---
    estatisticas_revenue = filmes_com_revenue['revenue'].describe()

    # Extraindo os valores do resultado do .describe()
    min_revenue = estatisticas_revenue['min']
    max_revenue = estatisticas_revenue['max']
    percentil_25 = estatisticas_revenue['25%']

    # Cálculo da receita total (o describe() não fornece a soma diretamente)
    total_revenue = filmes_com_revenue['revenue'].sum()

    print(f"Total da receita de todos os filmes: ${total_revenue:,.2f}")
    print(f"O maior valor de receita entre os filmes: ${max_revenue:,.2f}")
    print(f"O menor valor de receita entre os filmes: ${min_revenue:,.2f}")
    print(f"O valor do 25º percentil da receita: ${percentil_25:,.2f}")
    

    # --- Limpeza e Conversão da Coluna 'vote_count' ---
     # A coluna 'vote_count' também vem como texto e precisa ser convertida para número.
    # Usamos 'errors='coerce'' para transformar valores inválidos em NaN (Not a Number).
    df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com contagem de votos maior que zero.
    filmes_com_vote_count = df[df['vote_count'] > 0].copy()

    # --- Análise Estatística ---
    estatisticas_vote_count = filmes_com_vote_count['vote_count'].describe()

    # Extraindo os valores do resultado do .describe()
    min_vote_count = estatisticas_vote_count['min']
    max_vote_count = estatisticas_vote_count['max']
    percentil_75 = estatisticas_vote_count['75%']

    # Cálculo da receita total (o describe() não fornece a soma diretamente)
    total_vote_count = filmes_com_vote_count['vote_count'].sum()

    print(f"Total da contagem de votos de todos os filmes: {total_vote_count:,}")
    print(f"O maior valor de contagem de votos entre os filmes: {max_vote_count:,}")
    print(f"O menor valor de contagem de votos entre os filmes: {min_vote_count:,}")
    print(f"O valor do 75º percentil da contagem de votos: {percentil_75:,}")

except Exception as e:
    print(f"Ocorreu um erro durante a análise: {e}") 


Total do orçamento de todos os filmes: $192,062,026,597.00
Maior orçamento entre os filmes: $380,000,000.00
Menor orçamento entre os filmes: $1.00
Total da receita de todos os filmes: $509,576,984,595.00
O maior valor de receita entre os filmes: $2,787,965,087.00
O menor valor de receita entre os filmes: $1.00
O valor do 25º percentil da receita: $2,400,000.00
Total da contagem de votos de todos os filmes: 4,995,933.0
O maior valor de contagem de votos entre os filmes: 14,075.0
O menor valor de contagem de votos entre os filmes: 1.0
O valor do 75º percentil da contagem de votos: 38.0


## Análise de Categorias

10. Qual a `category` mais presente? E quantos valores únicos ela possui?(0,5)
11. Qual a `main_category` mais presente? E quantos valores únicos ela possui?(0,5)
12. Qual a `currency` mais presente? E quantos valores únicos ela possui?(0,5)
13. Execute o processo de dummy na coluna `country`.(0,5)

In [38]:
try:
    # Remove linhas com valores NaN na coluna 'genres'
    df.dropna(subset=['genres'], inplace=True)

    # Aplica a função para extrair os nomes dos gêneros
    def extrair_nomes_generos(generos_str):
        try:
            # Converte a string para uma lista de dicionários usando ast.literal_eval
            generos_lista = ast.literal_eval(generos_str)

            # Extrai os nomes dos gêneros
            nomes_generos = [genero['name'] for genero in generos_lista]
            return nomes_generos
        except (ValueError, SyntaxError):
            # Em caso de erro na conversão, retorna uma lista vazia
            return []
    
    # Aplica a função à coluna 'genres' e cria uma nova coluna 'generos_nomes'
    df['generos_nomes'] = df['genres'].apply(extrair_nomes_generos)


    # Cria uma lista com todos os gêneros presentes no DataFrame
    todos_os_generos = []
    for lista in df['generos_nomes']:
        todos_os_generos.extend(lista)

    # Conta a frequência de cada gênero
    contagem_generos = pd.Series(todos_os_generos).value_counts()

    # Obtém o gênero mais frequente e o número de gêneros únicos
    genero_mais_frequente = contagem_generos.index[0]
    
    # Número de gêneros únicos
    numero_generos_unicos = len(contagem_generos)

    print(f"O gênero mais frequente é: {genero_mais_frequente}")
    print(f"Número de gêneros únicos: {numero_generos_unicos}")


except Exception as e:
    print(f"Ocorreu um erro ao processar a coluna 'genres': {e}")


O gênero mais frequente é: Drama
Número de gêneros únicos: 32


In [35]:
df.head(100)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,NaN,3000000.0,"[{'id': 18, 'name': 'Drama'}]",NaN,406,tt0113247,fr,La Haine,Aimlessly whiling away their days in the concr...,...,1995-05-31,0.0,98.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Three Young Friends... One Last Chance.,La Haine,False,7.9,695.0
96,False,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45549,tt0111173,en,Shopping,"A dark, hip, urban story of a barren and anony...",...,1994-12-06,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one leaves without paying...,Shopping,False,5.6,13.0
97,False,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",http://www.nickbroomfield.com/heidifleiss.html,63076,tt0113283,en,Heidi Fleiss: Hollywood Madam,A documentary crew from the BBC arrives in L.A...,...,1995-12-27,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Heidi Fleiss: Hollywood Madam,False,6.8,4.0
98,False,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,11062,tt0115907,en,City Hall,The accidental shooting of a boy in New York l...,...,1996-02-16,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It started with a shootout on a rainswept stre...,City Hall,False,6.0,67.0
